(exploratory-data-analysis)=
# 탐색적 데이터 분석

## 소개

이 장에서는 데이터 과학자들이 줄여서 EDA라고 부르는 탐색적 데이터 분석이라는 체계적인 방식으로 데이터를 탐색하기 위해 시각화와 변환을 사용하는 방법을 보여줍니다. EDA는 반복적인 주기입니다. 당신은 다음을 수행합니다.

1.  데이터에 대한 질문을 생성합니다.

2.  데이터를 시각화, 변환 및 모델링하여 답변을 검색합니다.

3.  배운 내용을 사용하여 질문을 구체화하거나 새로운 질문을 생성합니다.

EDA는 엄격한 규칙 집합이 있는 공식적인 프로세스가 아니며 EDA의 초기 단계에서는 떠오르는 모든 아이디어를 자유롭게 조사해야 합니다. 이러한 아이디어 중 일부는 성공할 것이고 일부는 막다른 골목이 될 것입니다. 탐색을 계속하면서 결국에는 다른 사람들에게 작성하고 전달할 몇 가지 특히 생산적인 영역에 집중하게 될 것입니다. 데이터를 탐색할 때는 몇 가지 함정이 있다는 것을 기억해야 합니다. 데이터가 어떻게 수집되었는지, 무엇이 누락되었을 수 있는지, 품질 문제가 있는지 항상 생각해야 하며 상관 관계와 인과 관계의 차이점에 대해 매우 엄격해야 합니다(이것은 그 자체로 거대한 주제입니다!).

### 전제 조건

EDA를 수행하려면 **pandas**, **skimpy** 및 **pandas-profiling** 패키지를 사용합니다. 데이터 시각화를 위해 **lets-plot**도 필요합니다. 이 모든 패키지는 `pip install <패키지이름>`을 통해 설치할 수 있습니다.

항상 그렇듯이 사용할 패키지를 로드하는 것으로 시작합니다.

In [ ]:
import pandas as pd
from lets_plot import *
from lets_plot.mapping import as_discrete
from pandas.api.types import CategoricalDtype
from skimpy import skim

LetsPlot.setup_html()

In [ ]:
# 셀 제거
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline

# 플롯 설정
plt.style.use("https://github.com/aeturrell/python4DS/raw/main/plot_style.txt")
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

## 질문

> "일상적인 통계적 질문은 없으며 의심스러운 통계적 루틴만 있을 뿐입니다." --- 데이비드 콕스 경

> "종종 모호한 올바른 질문에 대한 근사적인 답변이 항상 정확하게 만들 수 있는 잘못된 질문에 대한 정확한 답변보다 훨씬 낫습니다." --- 존 튜키

EDA 중 목표는 데이터에 대한 이해를 발전시키는 것입니다. 이를 수행하는 가장 쉬운 방법은 질문을 도구로 사용하여 조사를 안내하는 것입니다. 질문을 하면 질문은 데이터 세트의 특정 부분에 주의를 집중시키고 어떤 그래프, 모델 또는 변환을 만들지 결정하는 데 도움이 됩니다.

EDA는 근본적으로 창의적인 과정입니다. 그리고 대부분의 창의적인 과정과 마찬가지로 *양질의* 질문을 하는 핵심은 많은 *양의* 질문을 생성하는 것입니다. 분석 시작 시에는 데이터 세트에 어떤 통찰력이 포함되어 있는지 알 수 없으므로 드러나는 질문을 하기가 어렵습니다. 반면에 새로운 질문을 할 때마다 데이터의 새로운 측면에 노출되고 발견 가능성이 높아집니다. 발견한 내용을 기반으로 각 질문에 새로운 질문을 추가하면 데이터의 가장 흥미로운 부분으로 빠르게 드릴다운하고 생각을 자극하는 질문 세트를 개발할 수 있습니다.

연구를 안내하기 위해 어떤 질문을 해야 하는지에 대한 규칙은 없습니다.
그러나 두 가지 유형의 질문은 항상 데이터 내에서 발견하는 데 유용합니다.
이러한 질문을 느슨하게 표현하면 다음과 같습니다.

1.  내 변수 내에서 어떤 유형의 변동이 발생합니까?

2.  내 변수 간에 어떤 유형의 공변동이 발생합니까?

이 장의 나머지 부분에서는 이 두 가지 질문을 살펴볼 것입니다.
변동과 공변동이 무엇인지 설명하고 각 질문에 답하는 여러 가지 방법을 보여 드리겠습니다.

## 변동

**변동**은 변수 값이 측정마다 변경되는 경향입니다. 실생활에서 변동을 쉽게 볼 수 있습니다. 연속 변수를 두 번 측정하면 두 가지 다른 결과가 나옵니다. 장비 결함으로 인해 빛의 속도와 같이 일정한 양을 측정하더라도 마찬가지입니다. 각 측정에는 측정마다 달라지는 약간의 오차가 포함됩니다. 다른 대상(예: 다른 사람의 눈 색깔) 또는 다른 시간(예: 다른 순간의 전자 에너지 수준)에 걸쳐 측정하는 경우에도 변수가 달라질 수 있습니다. 해당 패턴을 이해하는 가장 좋은 방법은 변수 값의 분포를 시각화하는 것입니다.

`diamonds` 데이터 세트에서 약 54,000개의 다이아몬드 무게(`"carat"`) 분포를 시각화하여 탐색을 시작하겠습니다.

In [ ]:
diamonds = pd.read_csv(
    "https://github.com/mwaskom/seaborn-data/raw/master/diamonds.csv"
)
diamonds["cut"] = diamonds["cut"].astype(
    CategoricalDtype(
        categories=["Fair", "Good", "Very Good", "Premium", "Ideal"], ordered=True
    )
)
diamonds["color"] = diamonds["color"].astype(
    CategoricalDtype(categories=["D", "E", "F", "G", "H", "I", "J"], ordered=True)
)
diamonds.head()

`"carat"`은 수치 변수이므로 히스토그램을 사용할 수 있습니다.

In [ ]:
(ggplot(diamonds, aes(x="carat")) + geom_histogram(binwidth=0.5))

이제 변동을 시각화할 수 있으므로 플롯에서 무엇을 찾아야 합니까?
그리고 어떤 유형의 후속 질문을 해야 합니까?
아래에는 그래프에서 찾을 수 있는 가장 유용한 정보 유형 목록과 각 정보 유형에 대한 몇 가지 후속 질문이 나와 있습니다.
좋은 후속 질문을 하는 핵심은 호기심(무엇에 대해 더 알고 싶습니까?)과 회의론(이것이 어떻게 오해의 소지가 있을 수 있습니까?)에 의존하는 것입니다.

### 일반적인 값

막대 차트와 히스토그램 모두에서 높은 막대는 변수의 일반적인 값을 보여주고 짧은 막대는 덜 일반적인 값을 보여줍니다.
막대가 없는 곳은 데이터에서 볼 수 없었던 값을 나타냅니다.
이 정보를 유용한 질문으로 바꾸려면 예상치 못한 것을 찾으십시오.

-   가장 일반적인 값은 무엇입니까?
    왜 그렇습니까?

-   드문 값은 무엇입니까?
    왜 그렇습니까?
    예상과 일치합니까?

-   특이한 패턴을 볼 수 있습니까?
    무엇이 그것들을 설명할 수 있습니까?

더 작은 다이아몬드의 `"carat"` 분포를 살펴보겠습니다.

`smaller_diamonds`를 만들 때 복사본을 만들어 수행합니다. 그렇지 않으면 `smaller_diamonds`에 대한 변경 사항이 `diamonds`에도 영향을 미칩니다(두 개는 컴퓨터 메모리의 동일한 기본 데이터를 가리킴). 때로는 컷이 원래 데이터 세트에 계속 연결되기를 원할 수도 있고 그렇지 않을 수도 있습니다. 이 경우 별개이기를 원하므로 `copy()`를 사용합니다.

In [ ]:
smaller_diamonds = diamonds.query("carat < 3").copy()

(ggplot(smaller_diamonds, aes(x="carat")) + geom_histogram(binwidth=0.01))

이 히스토그램은 몇 가지 흥미로운 질문을 제기합니다.

-   왜 전체 캐럿과 일반적인 분수 캐럿에 다이아몬드가 더 많습니까?

-   왜 각 피크의 약간 오른쪽보다 약간 왼쪽에 다이아몬드가 더 많습니까?

시각화는 또한 데이터에 하위 그룹이 있음을 시사하는 클러스터를 나타낼 수 있습니다.
하위 그룹을 이해하려면 다음을 질문하십시오.

-   각 하위 그룹 내의 관찰은 서로 어떻게 유사합니까?

-   별도의 클러스터에 있는 관찰은 서로 어떻게 다릅니까?

-   클러스터를 어떻게 설명하거나 설명할 수 있습니까?

-   클러스터의 모양이 오해의 소지가 있는 이유는 무엇입니까?

이러한 질문 중 일부는 데이터로 답변할 수 있지만 일부는 데이터에 대한 도메인 전문 지식이 필요합니다.
그중 많은 질문은 변수 간의 관계를 탐색하도록 유도합니다. 예를 들어 한 변수의 값이 다른 변수의 동작을 설명할 수 있는지 확인합니다.
곧 살펴보겠습니다.

### 특이값

이상값은 특이한 관찰입니다. 패턴에 맞지 않는 데이터 포인트입니다.
때로는 이상값이 데이터 입력 오류이거나, 때로는 이 데이터 수집에서 관찰된 극단값일 뿐이며, 다른 경우에는 중요한 새로운 발견을 시사합니다.
데이터가 많으면 히스토그램에서 이상값을 보기 어려울 때가 있습니다.
예를 들어 다이아몬드 데이터 세트에서 `"y"` 변수의 분포를 살펴보십시오.
이상값의 유일한 증거는 x축의 비정상적으로 넓은 한계입니다.


In [ ]:
(ggplot(diamonds, aes(x="y")) + geom_histogram(binwidth=0.5))

일반적인 구간에는 관찰이 너무 많아 드문 구간은 매우 짧아 보기 매우 어렵습니다(하지만 0을 열심히 쳐다보면 무언가를 발견할 수도 있습니다).
특이값을 쉽게 보려면 `coord_cartesian()`을 사용하여 y축의 작은 값으로 확대해야 합니다.

In [ ]:
(
    ggplot(diamonds, aes(x="y"))
    + geom_histogram(binwidth=0.5)
    + coord_cartesian(ylim=[0, 50])
)

`coord_cartesian()`에는 x축으로 확대해야 할 때 사용하는 `xlim()` 인수도 있습니다.
**Lets-Plot**에는 약간 다르게 작동하는 `xlim()` 및 `ylim()` 함수도 있습니다. 즉, 한계 밖의 데이터를 버립니다.

이를 통해 0, 약 30, 약 60의 세 가지 특이값을 볼 수 있습니다. **pandas**를 사용하여 이를 뽑아냅니다.

In [ ]:
unusual = diamonds.query("y < 3 or y > 20").loc[:, ["x", "y", "z", "price"]]
unusual

`"y"` 변수는 이 다이아몬드의 세 가지 치수 중 하나를 mm 단위로 측정합니다.
다이아몬드의 너비가 0mm일 수 없다는 것을 알고 있으므로 이러한 값은 잘못된 것입니다.
EDA를 수행하여 `NA`를 단순히 검색해서는 절대 찾을 수 없었던 0으로 코딩된 누락된 데이터를 발견했습니다.
앞으로는 오해의 소지가 있는 계산을 방지하기 위해 이러한 값을 `NA`로 다시 코딩하도록 선택할 수 있습니다.
또한 32mm 및 59mm 측정값이 타당하지 않다고 의심할 수 있습니다. 해당 다이아몬드는 길이가 1인치가 넘지만 수십만 달러가 들지 않습니다!

이상값을 포함하거나 제외하여 분석을 반복하는 것이 좋습니다.
결과에 미미한 영향을 미치고 왜 거기에 있는지 알 수 없다면 생략하고 계속 진행하는 것이 합리적입니다.
그러나 결과에 상당한 영향을 미치는 경우 정당화 없이 삭제해서는 안 됩니다.
원인(예: 데이터 입력 오류)을 파악하고 보고서에 삭제했음을 공개해야 합니다.


### 연습

1.  `diamonds`에서 `x`, `y`, `z` 각 변수의 분포를 탐색합니다.
    무엇을 배웁니까?
    다이아몬드를 생각하고 어떤 차원이 길이, 너비, 깊이인지 어떻게 결정할지 생각해 보십시오.

2.  `"price"`의 분포를 탐색합니다.
    특이하거나 놀라운 것을 발견합니까?
    (힌트: `binwidth=` 키워드 인수 설정을 신중하게 생각하고 다양한 값을 시도해 보십시오.)

3.  0.99캐럿 다이아몬드는 몇 개입니까?
    1캐럿 다이아몬드는 몇 개입니까?
    차이의 원인이 무엇이라고 생각합니까?

4.  히스토그램을 확대할 때 `coord_cartesian()`과 `xlim()` 또는 `ylim()`을 비교하고 대조합니다.
    binwidth를 설정하지 않으면 어떻게 됩니까?
    막대의 절반만 표시하도록 확대하려고 하면 어떻게 됩니까?


## 특이값

데이터 세트에서 특이값을 발견했고 단순히 나머지 분석으로 넘어가고 싶다면 두 가지 옵션이 있습니다.

1.  이상한 값이 있는 전체 행을 삭제합니다.

    ```파이썬
    condition = ((diamonds["y"] < 3) | (diamonds["y"] > 20))
    diamonds2 = diamonds.loc[~condition, :]
    ```

    하나의 잘못된 값이 해당 관찰에 대한 다른 모든 값도 잘못되었음을 의미하지는 않으므로 이 옵션은 권장하지 않습니다.
    또한 데이터 품질이 낮은 경우 모든 변수에 이 접근 방식을 적용하면 데이터가 전혀 남지 않을 수 있습니다!

2.  대신 특이값을 누락된 값으로 바꾸는 것이 좋습니다.
    이를 수행하는 한 가지 방법은 특이값이 바뀐 데이터 프레임과 원래 데이터를 구분하는 복사본을 만든 다음 문제가 있는 값을 **pandas**의 특수 NA 값인 `pd.NA`로 설정하는 것입니다.


In [ ]:
diamonds2 = diamonds.copy()
condition = (diamonds2["y"] < 3) | (diamonds2["y"] > 20)
diamonds2.loc[condition, "y"] = pd.NA

누락된 값을 어디에 플로팅해야 하는지 명확하지 않으므로 **lets-plot**은 플롯에 포함하지 않습니다.

In [ ]:
(ggplot(diamonds2, aes(x="x", y="y")) + geom_point())

다른 경우에는 누락된 값이 있는 관찰과 기록된 값이 있는 관찰을 다르게 만드는 요인을 이해하고 싶을 수 있습니다.
예를 들어 nycflights13 데이터에서 `"dep_time"` 변수의 누락된 값은 항공편이 취소되었음을 나타냅니다.
따라서 취소된 항공편과 취소되지 않은 항공편의 예정된 출발 시간을 비교하고 싶을 수 있습니다.
`is.na()`를 사용하여 `"dep_time"`이 누락되었는지 확인하여 새 변수를 만들어 이 작업을 수행할 수 있습니다.

In [ ]:
url = "https://raw.githubusercontent.com/byuidatascience/data4python4ds/master/data-raw/flights/flights.csv"
flights = pd.read_csv(url)
flights.head()

In [ ]:
flights2 = flights.assign(
    cancelled=lambda x: pd.isna(x["dep_time"]),
    sched_hour=lambda x: x["sched_dep_time"] // 100,
    sched_min=lambda x: x["sched_dep_time"] % 100,
    sched_dep_time=lambda x: x["sched_hour"] + x["sched_min"] / 60,
)

(
    ggplot(flights2, aes(x="sched_dep_time"))
    + geom_freqpoly(aes(color="cancelled"), binwidth=1 / 4)
)

그러나 이 플롯은 취소되지 않은 항공편이 취소된 항공편보다 훨씬 많기 때문에 그다지 좋지 않습니다.
다음 섹션에서는 이 비교를 개선하기 위한 몇 가지 기술을 살펴보겠습니다.

### 연습

1.  히스토그램에서 누락된 값은 어떻게 됩니까?
    막대 차트에서 누락된 값은 어떻게 됩니까?
    히스토그램과 막대 차트에서 누락된 값을 처리하는 방식에 차이가 있는 이유는 무엇입니까?

2.  항공편이 취소되었는지 여부로 색칠된 `scheduled_dep_time`의 빈도 플롯을 다시 만듭니다.
    또한 `cancelled` 변수로 패싯합니다.
    취소된 항공편보다 취소되지 않은 항공편이 더 많은 효과를 완화하기 위해 패싯 함수의 `scales` 변수의 다른 값으로 실험합니다.


## 공분산

변동이 변수 *내의* 동작을 설명한다면 공분산은 변수 *간의* 동작을 설명합니다.
**공분산**은 두 개 이상의 변수 값이 관련된 방식으로 함께 변동하는 경향입니다.
공분산을 발견하는 가장 좋은 방법은 두 개 이상의 변수 간의 관계를 시각화하는 것이지만 공분산이 변수 간의 인과 관계를 의미하지는 않는다는 점에 유의하십시오.

### 범주형 변수와 수치형 변수

예를 들어 `geom_freqpoly()`를 사용하여 다이아몬드 가격이 품질(`"cut"`로 측정)에 따라 어떻게 달라지는지 살펴보겠습니다.


In [ ]:
(
    ggplot(diamonds, aes(x="price"))
    + geom_freqpoly(aes(color="cut"), binwidth=500, linewidth=0.75)
)

`geom_freqpoly()`의 기본 모양은 전체 개수로 결정되는 높이가 컷마다 너무 많이 달라 분포 모양의 차이를 보기 어렵기 때문에 여기서는 그다지 유용하지 않습니다.

비교를 더 쉽게 하려면 y축에 표시되는 내용을 바꿔야 합니다.
개수 대신 각 빈도 다각형 아래 영역이 1이 되도록 표준화된 개수인 **밀도**를 표시합니다.

In [ ]:
(
    ggplot(diamonds, aes(x="price"))
    + geom_density(aes(color="cut", fill="cut"), size=1, alpha=0.2)
)

이 플롯에는 다소 놀라운 점이 있습니다. 가장 낮은 품질인 페어 다이아몬드가 평균 가격이 가장 높은 것으로 보입니다! 하지만 밀도 플롯은 해석하기가 약간 어렵기 때문일 수 있습니다. 이 플롯에는 많은 내용이 담겨 있습니다.

이 관계를 탐색하는 데 시각적으로 더 간단한 플롯은 나란히 놓인 상자 그림을 사용하는 것입니다.

In [ ]:
(ggplot(diamonds, aes(x="cut", y="price")) + geom_boxplot())

분포에 대한 정보는 훨씬 적게 보이지만 상자 그림은 훨씬 더 간결하므로 더 쉽게 비교할 수 있습니다(그리고 한 플롯에 더 많이 맞출 수 있음). 더 나은 품질의 다이아몬드가 일반적으로 더 저렴하다는 직관에 반하는 결과를 뒷받침합니다! 연습 문제에서는 그 이유를 알아내도록 도전할 것입니다.

`"cut"`은 순서형 범주 변수입니다. 페어는 굿보다 나쁘고, 굿은 베리 굿보다 나쁩니다. 많은 범주형 변수에는 이러한 본질적인 순서가 없으므로 더 유익한 표시를 위해 순서를 변경하고 싶을 수 있습니다. 한 가지 방법은 중앙값을 기준으로 하는 것이지만 다른 옵션도 사용할 수 있습니다.

mpg 데이터 세트를 예로 들어 고속도로 주행 거리가 클래스에 따라 어떻게 달라지는지 살펴볼 수 있습니다.

In [ ]:
mpg = pd.read_csv(
    "https://vincentarelbundock.github.io/Rdatasets/csv/ggplot2/mpg.csv", index_col=0
)
mpg["class"] = mpg["class"].astype("category")

(ggplot(mpg, aes(x="class", y="hwy")) + geom_boxplot())

추세를 더 쉽게 보려면 `"hwy"`의 중앙값을 기준으로 클래스 순서를 변경할 수 있습니다.

In [ ]:
(ggplot(mpg) + geom_boxplot(aes(as_discrete("class", order_by="..middle.."), "hwy")))

변수 이름이 길면 geom_boxplot()을 90° 뒤집으면 더 잘 작동합니다. `coord_flip()`을 추가하여 이 작업을 수행할 수 있습니다.

In [ ]:
(
    ggplot(mpg)
    + geom_boxplot(aes(as_discrete("class", order_by="..middle.."), "hwy"))
    + coord_flip()
)

#### 연습

1.  배운 내용을 사용하여 취소된 항공편과 취소되지 않은 항공편의 출발 시간 시각화를 개선합니다.

2.  EDA를 기반으로 다이아몬드 데이터 세트에서 다이아몬드 가격을 예측하는 데 가장 중요한 변수는 무엇입니까?
    해당 변수는 컷과 어떤 상관 관계가 있습니까?
    이 두 관계의 조합이 품질이 낮은 다이아몬드가 더 비싼 이유를 설명하는 이유는 무엇입니까?

3.  `geom_violin()`을 사용하여 `diamonds` 데이터 세트의 범주형 변수와 다이아몬드 가격을 시각화한 다음 패싯된 `geom_histogram()`, 색칠된 `geom_freqpoly()`, 색칠된 `geom_density()`를 사용합니다.
    네 가지 플롯을 비교하고 대조합니다.
    범주형 변수의 수준을 기반으로 수치 변수의 분포를 시각화하는 각 방법의 장단점은 무엇입니까?

4.  데이터 세트가 작은 경우 `geom_jitter()`를 사용하여 과도한 플로팅을 방지하여 연속 변수와 범주형 변수 간의 관계를 더 쉽게 볼 수 있습니다.
    ggbeeswarm 패키지는 `geom_jitter()`와 유사한 여러 가지 방법을 제공합니다.
    나열하고 각 방법이 수행하는 작업을 간략하게 설명합니다.

### 두 개의 범주형 변수

범주형 변수 간의 공변동을 시각화하려면 이러한 범주형 변수의 각 수준 조합에 대한 관찰 수를 계산해야 합니다. 그런 다음 "깔끔한" 형식으로 만들기 위해 녹이는 `pd.crosstab()`을 사용하여 이 작업을 수행할 수 있습니다.

In [ ]:
ct_cut_color = pd.melt(
    pd.crosstab(diamonds["cut"], diamonds["color"]).reset_index(),
    id_vars=["cut"],
    value_vars=diamonds["color"].unique(),
)

`geom_tile()`로 시각화합니다.

In [ ]:
(ggplot(ct_cut_color, aes(x="color", y="cut")) + geom_tile(aes(fill="value")))

### 연습

1. 위 개수 데이터 세트를 어떻게 다시 조정하여 색상 내 컷 또는 컷 내 색상의 분포를 더 명확하게 보여줄 수 있습니까?

2. 색상이 `x` 미학에 매핑되고 컷이 채우기 미학에 매핑된 경우 분할 막대 차트에서 어떤 다른 데이터 통찰력을 얻을 수 있습니까? 각 세그먼트에 속하는 개수를 계산합니다.

3. **pandas**와 함께 `geom_tile()`을 사용하여 평균 항공편 출발 지연이 목적지 및 연중 월에 따라 어떻게 달라지는지 탐색합니다. 플롯을 읽기 어렵게 만드는 것은 무엇입니까? 어떻게 개선할 수 있습니까?

### 두 개의 수치형 변수

두 수치형 변수 간의 공변동을 시각화하는 한 가지 좋은 방법인 `geom_point()`로 산점도를 그리는 것을 이미 보았습니다.
점의 패턴으로 공변동을 볼 수 있습니다.
예를 들어 다이아몬드의 캐럿 크기와 가격 간에 양의 연관성을 볼 수 있습니다. 캐럿이 더 많은 다이아몬드는 가격이 더 높습니다.
관계는 지수적입니다.

In [ ]:
(ggplot(smaller_diamonds, aes(x="carat", y="price")) + geom_point())

(이 섹션에서는 3캐럿보다 작은 다이아몬드의 대부분에 집중하기 위해 `smaller_diamonds` 데이터 세트를 사용합니다.)

데이터 세트 크기가 커짐에 따라 산점도는 덜 유용해집니다. 점이 겹쳐지고 균일한 검은색 영역으로 쌓여 2차원 공간 전체에서 데이터 밀도의 차이를 판단하기 어렵고 추세를 파악하기도 어렵습니다.
문제를 해결하는 한 가지 방법인 `alpha` 미학을 사용하여 투명도를 추가하는 것을 이미 보았습니다.


In [ ]:
(ggplot(smaller_diamonds, aes(x="carat", y="price")) + geom_point(alpha=1 / 20))

그러나 매우 큰 데이터 세트의 경우 투명도를 사용하는 것이 어려울 수 있습니다. 이 경우 *구간 산점도* 또는 구간화된 산점도를 권장합니다. 구간화된 산점도는 조건 변수(이 예에서는 `"carat"`)를 동일한 크기의 구간 또는 분위수로 나누고 각 구간 내 종속 변수(이 예에서는 `"price"`)의 조건부 평균을 플로팅합니다. 구간 산점도에는 종종 신뢰 구간도 함께 제공됩니다. 파이썬의 좋은 구간 산점도 패키지는 [**binsreg**](https://nppackages.github.io/binsreg/)입니다. 그러나 구간 산점도는 고급 주제이므로 여기서는 다루지 않습니다.

## EDA를 위한 **pandas** 내장 도구

**pandas**에는 EDA를 위한 훌륭한 내장 옵션이 있습니다. 실제로 이미 그중 하나인 `df.info()`를 보았습니다. 이 함수는 데이터 유형과 메모리 사용량을 보고하는 것 외에도 각 열의 관찰 중 '참 같은' 값과 '거짓 같은' 값의 개수, 즉 null이 아닌 값의 개수를 알려줍니다.

### 탐색적 표 및 기술 통계

표를 얻기 위해 `.info()`를 넘어서는 작은 단계는 `.describe()`를 사용하는 것입니다. 부동 소수점을 포함한 혼합 데이터 유형이 있는 경우 몇 가지 기본 요약 통계를 보고합니다.

In [ ]:
diamonds.describe()

도움이 되지만 읽기가 확실히 어렵습니다! `round()` 메서드를 사용하여 이를 개선할 수 있습니다.

In [ ]:
sum_table = diamonds.describe().round(1)
sum_table

게시된 요약 통계 표에는 종종 행당 하나의 변수가 나열되며 데이터 프레임에 변수가 많은 경우 `describe()`가 너무 넓어져 쉽게 읽을 수 없게 될 수 있습니다. `T` 속성(또는 `transpose()` 메서드)을 사용하여 전치할 수 있습니다.

In [ ]:
sum_table = sum_table.T
sum_table

물론 이 미리 만들어진 표에 제공된 통계는 그다지 사용자 정의되지 않았습니다. 그렇다면 실제로 원하는 표를 얻으려면 어떻게 해야 할까요? 글쎄요, 답은 이전 데이터 장, 특히 데이터 분석 소개의 내용을 활용하는 것입니다. 그룹화, 병합, 집계: 원하는 EDA 표를 만들기 위해 이 모든 것을 사용하십시오.

데이터를 탐색하는 경우 모든 것을 명확하게 읽고 예상과 다른 편차를 빠르게 확인할 수 있기를 원할 수도 있습니다. **pandas**에는 이를 지원하는 몇 가지 내장 기능이 있습니다. 이러한 스타일은 데이터 프레임을 예를 들어 Excel로 내보낼 때도 유지됩니다.

다음은 데이터 프레임 스타일 지정의 몇 가지 방법을 보여주는 예입니다. 더 넓은 형식으로 풀기(`unstack`), 단위 변경(`lambda` 함수, 참고: `1e3`은 컴퓨터에서 `1000`의 약자임), NaN을 눈에 띄지 않는 문자열로 채우기(`.fillna('-')`), 소수점 이하 숫자 제거(`.style.format(precision=0)`), 캡션 추가(`.style.set_caption`)와 같은 여러 기능을 사용합니다.

In [ ]:
(
    diamonds.groupby(["cut", "color"])["price"]
    .mean()
    .unstack()
    .apply(lambda x: x / 1e3)
    .fillna("-")
    .style.format(precision=2)
    .set_caption("판매 가격 (천 단위)")
)

우리가 본 것보다 깔끔하지만 이것은 여전히 단조로운 숫자 표입니다. 눈에 바로 띄지 않습니다!

이를 해결하기 위해 또 다른 스타일링 기법인 색상 사용을 살펴보겠습니다. 컷과 색상 간의 교차 tabulation, 즉 범주에 따라 이러한 필드 모두에 나타나는 객체 수를 보여주는 표를 만들고 싶다고 가정해 보겠습니다.

교차 tabulation을 수행하려면 내장된 `pd.crosstab()`을 사용하지만 `style.background_gradient()`를 사용하여 표에 나타나는 값(개수)을 히트맵으로 강조 표시하도록 요청합니다.

In [ ]:
pd.crosstab(diamonds["color"], diamonds["cut"]).style.background_gradient(cmap="plasma")

기본적으로 `background_gradient()`는 각 숫자를 해당 열의 다른 숫자와 비교하여 강조 표시합니다. `axis=1`을 사용하여 행별로 강조 표시하거나 `axis=0`을 사용하여 모든 표 값과 비교하여 강조 표시할 수 있습니다. 그리고 물론 `plasma`는 [사용 가능한 많은 색상 맵](https://matplotlib.org/stable/tutorials/colors/colormaps.html) 중 하나일 뿐입니다!

```{admonition} 연습
다른 색상 맵을 사용하여 새 교차 tabulation을 수행합니다.
```

다음은 데이터 프레임에 대한 몇 가지 다른 스타일링 팁입니다.

먼저 막대를 사용하여 순서를 표시합니다.

In [ ]:
(
    pd.crosstab(diamonds["color"], diamonds["cut"])
    .style.format(precision=0)
    .bar(color="#d65f5f")
)

`.hightlight_max()` 및 유사한 명령을 사용하여 중요한 항목을 표시합니다.

In [ ]:
pd.crosstab(diamonds["color"], diamonds["cut"]).style.highlight_max().format("{:.0f}")

[여기](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html#Styling)에서 전체 스타일링 명령 집합을 찾을 수 있습니다.

### **pandas**를 사용한 탐색적 플로팅

**pandas**에는 데이터를 빠르게 살펴보는 데 도움이 되는 몇 가지 내장 플로팅 옵션이 있습니다. 컨텍스트에 따라 `.plot.*` 또는 `.plot()`를 통해 액세스할 수 있습니다. 택시에 대한 데이터 세트를 사용하여 빠른 `.plot()`을 만들어 보겠습니다.

In [ ]:
taxis = pd.read_csv("https://github.com/mwaskom/seaborn-data/raw/master/taxis.csv")
# 픽업 시간 열을 datetime으로 변환합니다.
taxis["pickup"] = pd.to_datetime(taxis["pickup"])
# 다른 열 유형을 설정합니다.
taxis = taxis.astype(
    {
        "dropoff": "datetime64[ns]",
        "pickup": "datetime64[ns]",
        "color": "category",
        "payment": "category",
        "pickup_zone": "string",
        "dropoff_zone": "string",
        "pickup_borough": "category",
        "dropoff_borough": "category",
    }
)
taxis.head()

In [ ]:
taxis.info()

In [ ]:
(
    taxis.set_index("pickup")
    .groupby(pd.Grouper(freq="D"))["total"]
    .mean()
    .plot(
        title="평균 택시 요금",
        xlabel="",
        ylabel="요금 (USD)",
    )
);

다시 말하지만, 데이터를 올바른 형태로 만들면 플로팅할 수 있습니다. 동일한 함수가 여러 줄에서 작동합니다.

In [ ]:
(
    taxis.set_index("pickup")
    .groupby(pd.Grouper(freq="D"))[["fare", "tip", "tolls"]]
    .mean()
    .plot(
        style=["-", ":", "-."],
        title="택시 요금 구성 요소",
        xlabel="",
        ylabel="USD",
    )
);

이제 다른 빠른 `.plot.*` 옵션을 살펴보겠습니다.

막대 차트(가로 방향의 경우 `barh` 사용, `rot`은 레이블 회전 설정):

In [ ]:
taxis.value_counts("payment").sort_index().plot.bar(title="개수", rot=0);

다음은 `.plot.hist()`를 사용하여 히스토그램을 만듭니다.

In [ ]:
taxis["tip"].plot.hist(bins=30, title="팁");

상자 그림:

In [ ]:
(taxis[["fare", "tolls", "tip"]].plot.box());

산점도:

In [ ]:
taxis.plot.scatter(x="fare", y="tip", alpha=0.7, ylim=(0, None));

## EDA를 위한 기타 도구

**pandas**와 시각화 패키지 사이에는 EDA에 필요한 많은 것이 있습니다. 하지만 EDA를 더 쉽게 만드는 데만 전념하는 몇 가지 도구가 있으며 알아두면 좋습니다.

### 요약 통계를 위한 **skimpy**

**skimpy** 패키지는 콘솔의 데이터 프레임 변수에 대한 요약 통계를 제공하는 경량 도구입니다(이 장의 나머지 부분에 있는 다른 EDA 패키지가 수행하는 큰 HTML 보고서가 아님). 때로는 데이터 프레임에서 `.summary()`를 실행하는 것만으로는 충분하지 않으며 **skimpy**가 이 격차를 메웁니다. 또한 이전 장에서 본 열 이름을 정리하기 위한 `clean_columns()` 함수도 함께 제공됩니다. **skimpy**를 설치하려면 터미널에서 `pip install skimpy`를 실행합니다.

**skimpy**를 실제로 살펴보겠습니다.

In [ ]:
skim(taxis)

## 요약

이 장에서는 데이터 내 변동을 이해하는 데 도움이 되는 다양한 도구를 배웠습니다.
한 번에 단일 변수로 작업하는 기술과 변수 쌍으로 작업하는 기술을 보았습니다.
데이터에 수십 또는 수백 개의 변수가 있는 경우 이는 고통스러울 정도로 제한적으로 보일 수 있지만 다른 모든 기술이 구축되는 기초입니다.

다음 장에서는 결과를 전달하는 데 사용할 수 있는 도구에 중점을 둘 것입니다.